# TFT univariate experiments

This notebook contains the experiments for TFT on univariate load, solar and wind prediction.
1. Contrary, to Informer, Autoformer and DLinear implementation, we manually need to scale and split the datasets.
2. Add time features such as month of the year, day of week etc..
3. Implement an experiment_main() method that runs the whole model training across datasets and prediction lengths
4. Wrap the data in the required format by pytorch-forecasting and set up the model
5. We calculate the results with a rolling test data set

In [146]:
import copy

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger

import numpy as np
import os
import pandas as pd
from pathlib import Path
import pickle

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import EncoderNormalizer
from pytorch_forecasting.data import MultiNormalizer
from pytorch_forecasting.data import TorchNormalizer
from pytorch_forecasting.metrics import MAE, RMSE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

from sklearn.preprocessing import StandardScaler
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')

## 1. Load, Scale, Split Data

In [382]:
file_path = '../../01_dataset/Open-Power-System/df_most_important_columns.csv'
df = pd.read_csv(file_path)

In [383]:
df

,date,DE_load_actual_entsoe_transparency,DE_solar_generation_actual,DE_wind_generation_actual
0,2015-01-01 07:00:00,41133.0,71.0,10208.0
1,2015-01-01 08:00:00,42963.0,773.0,10029.0
2,2015-01-01 09:00:00,45088.0,2117.0,10550.0
3,2015-01-01 10:00:00,47013.0,3364.0,11390.0
4,2015-01-01 11:00:00,48159.0,4198.0,12103.0
...,...,...,...,...
43788,2019-12-30 19:00:00,53959.0,0.0,32323.0
43789,2019-12-30 20:00:00,51937.0,0.0,32395.0
43790,2019-12-30 21:00:00,50574.0,0.0,31439.0
43791,2019-12-30 22:00:00,47382.0,0.0,30727.0


In [384]:
def preprocess_data(df):
 
    # Extract the date column
    dates = df['date']
    
    # Exclude the date column for scaling
    df = df.drop(columns=['date'])
    
    # Define the sizes for training, validation, and test sets (70%,10%,20%)
    train_size = int(len(df) * 0.7)
    val_size = int((len(df) * 0.1)+1)
    test_size = len(df) - train_size - val_size
    
    # Split the data into training, validation, and test set
    train_data = df.iloc[:train_size]
    val_data = df.iloc[train_size:train_size + val_size]
    test_data = df.iloc[train_size + val_size:]
    
    # Initialize the StandardScaler
    scaler = StandardScaler()
    
    # Fit the scaler on the training data
    scaler.fit(train_data)
    
    # Transform the datasets using the same scaler
    train_standardized = scaler.transform(train_data)
    val_standardized = scaler.transform(val_data)
    test_standardized = scaler.transform(test_data)
    
    # Create new DataFrames with standardized values, including the date column
    train_data = pd.DataFrame(train_standardized, columns=train_data.columns)
    val_data = pd.DataFrame(val_standardized, columns=df.columns)
    test_data = pd.DataFrame(test_standardized, columns=test_data.columns)
    
    # Add the date column back to the data
    train_data['date'] = dates.iloc[:train_size].values
    val_data['date'] = dates.iloc[train_size:train_size + val_size].values
    test_data['date'] = dates.iloc[train_size + val_size:].values
     
    # Set 'date' column as index and convert it to datetime format
    train_data['date'] = pd.to_datetime(train_data['date'])
    train_data.set_index('date', inplace=True)

    test_data['date'] = pd.to_datetime(test_data['date'])
    test_data.set_index('date', inplace=True)
    
    val_data['date'] = pd.to_datetime(val_data['date'])
    val_data.set_index('date', inplace=True)
 
    return train_data, val_data, test_data

In [385]:
train_data_all_columns, val_data_all_columns, test_data_all_columns = preprocess_data(df)

In [166]:
# This is needed because TFT requires one dataset as an input and additional information on where to split up between training and validation
df = pd.concat([train_data_all_columns, val_data_all_columns], ignore_index=False)

## 2. Time Features

In [167]:
def add_time_features(df):
    # Convert the index to a DateTimeIndex if it's not already
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    
    # Extract the desired date components
    hour_of_day = df.index.hour.astype(str).astype("category")
    day_of_month = df.index.day.astype(str).astype("category")
    day_of_year = df.index.dayofyear.astype(str).astype("category")
    month_of_year = df.index.month.astype(str).astype("category")
    week_of_year = df.index.isocalendar().week.astype(str).astype("category")
    day_of_week = df.index.dayofweek.astype(str).astype("category")  # Monday is 0, Sunday is 6
    # Append these Series as new columns in the DataFrame
    df = df.assign(
        hour_of_day=hour_of_day,
        day_of_month=day_of_month,
        day_of_year=day_of_year,
        month_of_year=month_of_year,
        week_of_year=week_of_year,
        day_of_week=day_of_week
    )
    return df

In [168]:
df = add_time_features(df)

In [170]:
# Reset the index for df
df = df.reset_index(drop=True)
# A time_idx is required by TFT
df['time_idx']=df.index
# TFT requires at least on category for prediction. Since we only have one category/one country we add 'DE' to every row
df['DE'] = 'DE'

## 3. Experiment Method

In [179]:
def experiment_main():
    # Define the save directory
    save_dir = 'TFT_Results'  

    # Create the directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    # Instead of starting each experiment manually we start the experiments once and loop through the different combinations
    columns = ["DE_load_actual_entsoe_transparency", "DE_solar_generation_actual", "DE_wind_generation_actual"]
    prediction_lengths = [24, 48, 96, 192]
    for column in columns:
        for pred_len in prediction_lengths:
            dataset_builder_model_settings(df[[column,'time_idx','DE','hour_of_day','day_of_month','day_of_year','month_of_year','week_of_year','day_of_week']], pred_len, column)

## 4. Define the Dataset Characteristics as required by TFT and set the model parameters

In [183]:
def dataset_builder_model_settings(data, pred_len, columns):
    
    # Sets the maximum encoder length same across models
    max_encoder_length = 96
    # Ensures the validation data is not used for training
    training_cutoff = data["time_idx"].max() - 4379
    
    # Define the training dataset in the TimeSeriesDataSet format
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx < training_cutoff],
        time_idx="time_idx",
        target=columns,
        group_ids=["DE"],
        min_encoder_length=max_encoder_length,  
        max_encoder_length=max_encoder_length,
        min_prediction_length=pred_len,
        max_prediction_length=pred_len,
        static_categoricals=["DE"],
        time_varying_known_categoricals=['hour_of_day','day_of_month','day_of_year','month_of_year','week_of_year','day_of_week'],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[columns],
        # Pytorch-forecasting requires a target_normalizer. However, the data is already normalized to use the same normalization across models. Therefore, this normalizer does not change the data.  
        target_normalizer=TorchNormalizer(method='identity', center=False, transformation=None, method_kwargs={}),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
    )
    print(training)
  
    # Define validation dataset with rolling validation approach
    validation = TimeSeriesDataSet.from_dataset(training, data, predict=False, min_prediction_idx=data["time_idx"].max() - 4379, predict_mode=False, stop_randomization=True)
    print(validation)
    
    # Create dataloaders for model
    batch_size = 128  # Set this between 32 to 128
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    
    # Configure network and trainer
    pl.seed_everything(42)
    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=50,
        accelerator="gpu",
        enable_model_summary=True,
        gradient_clip_val=0.1,
        limit_train_batches=50,  
        callbacks=[lr_logger, early_stop_callback],
        logger=logger,
    )

    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.03,
        hidden_size=16,
        attention_head_size=2,
        dropout=0.1,
        hidden_continuous_size=8,
        loss=MAE(),
        log_interval=10, 
        optimizer="adam",
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")
    
     # Fit network
    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )
    
    # Define the folder where you want to save the information
    output_folder = "TFT_Results"

    # Define the file path within the folder
    output_file = os.path.join(output_folder, "model_info_univariate.txt")

    # Create a dictionary to store data
    data = {"pred_len": pred_len, "num_columns": columns, "best_model_path": trainer.checkpoint_callback.best_model_path}
    
    # Save the data to the file
    with open(output_file, "a") as file:
        file.write(str(data) + "\n")
                               
    # Print the saved information
    print("Model information saved to", output_file)

In [184]:
experiment_main()

Seed set to 42
Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


TimeSeriesDataSet[length=30536](
	time_idx='time_idx',
	target='DE_load_actual_entsoe_transparency',
	group_ids=['DE'],
	weight=None,
	max_encoder_length=96,
	min_encoder_length=96,
	min_prediction_idx=0,
	min_prediction_length=24,
	max_prediction_length=24,
	static_categoricals=['DE'],
	static_reals=['encoder_length', 'DE_load_actual_entsoe_transparency_center', 'DE_load_actual_entsoe_transparency_scale'],
	time_varying_known_categoricals=['hour_of_day', 'day_of_month', 'day_of_year', 'month_of_year', 'week_of_year', 'day_of_week'],
	time_varying_known_reals=['time_idx', 'relative_time_idx'],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['DE_load_actual_entsoe_transparency'],
	variable_groups={},
	constant_fill_strategy={},
	allow_missing_timesteps=False,
	lags={},
	add_relative_time_idx=True,
	add_target_scales=True,
	add_encoder_length=True,
	target_normalizer=TorchNormalizer(method='identity', center=False, transformation=None, method_kwargs={}),
	categorical_

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | MAE                             | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 7.3 K 
3  | prescalers                         | ModuleDict                      | 96    
4  | static_variable_selection          | VariableSelectionNetwork        | 1.8 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 3.1 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 2.4 K 
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K 
9  | static_context_initial_cell_l

Epoch 0: 100%|██████████| 50/50 [00:18<00:00,  2.67it/s, v_num=27, train_loss_step=0.214]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 50/50 [00:18<00:00,  2.71it/s, v_num=27, train_loss_step=0.207, val_loss=0.210, train_loss_epoch=0.455]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 50/50 [00:18<00:00,  2.63it/s, v_num=27, train_loss_step=0.161, val_loss=0.175, train_loss_epoch=0.192]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 50/50 [00:18<00:00,  2.73it/s, v_num=27, train_loss_step=0.128, val_loss=0.187, train_loss_epoch=0.170]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 50/50 [00:18<00:00,  2.73it/s, v_num=27, train_loss_step=0.120, val_loss=0.160, train_loss_epoch=0.151]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 50/50 [00:18<00:00,  2.74it/s, v_num=27, train_loss_step=0.148, val_loss=0.160, train_loss_epoch=0.137]
Validation: |          | 0/? [

Seed set to 42
Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

Epoch 0: 100%|██████████| 50/50 [00:24<00:00,  2.04it/s, v_num=37, train_loss_step=0.513]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 50/50 [00:24<00:00,  2.05it/s, v_num=37, train_loss_step=0.413, val_loss=0.867, train_loss_epoch=0.653]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 50/50 [00:24<00:00,  2.07it/s, v_num=37, train_loss_step=0.360, val_loss=0.946, train_loss_epoch=0.481]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 50/50 [00:24<00:00,  2.07it/s, v_num=37, train_loss_step=0.305, val_loss=1.050, train_loss_epoch=0.382]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 50/50 [00:24<00:00,  2.06it/s, v_num=37, train_loss_step=0.286, val_loss=1.020, train_loss_epoch=0.324]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 50/50 [00:23<00:00,  2.10it/s, v_num=37, train_loss_step=0.252, val_loss=1.000, train_loss_epoch=0.293]
Validation: |          | 0/? [

Seed set to 42
Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


TimeSeriesDataSet[length=4189](
	time_idx='time_idx',
	target='DE_wind_generation_actual',
	group_ids=['DE'],
	weight=None,
	max_encoder_length=96,
	min_encoder_length=96,
	min_prediction_idx=30655,
	min_prediction_length=192,
	max_prediction_length=192,
	static_categoricals=['DE'],
	static_reals=['encoder_length', 'DE_wind_generation_actual_center', 'DE_wind_generation_actual_scale'],
	time_varying_known_categoricals=['hour_of_day', 'day_of_month', 'day_of_year', 'month_of_year', 'week_of_year', 'day_of_week'],
	time_varying_known_reals=['time_idx', 'relative_time_idx'],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['DE_wind_generation_actual'],
	variable_groups={},
	constant_fill_strategy={},
	allow_missing_timesteps=False,
	lags={},
	add_relative_time_idx=True,
	add_target_scales=True,
	add_encoder_length=True,
	target_normalizer=TorchNormalizer(method='identity', center=False, transformation=None, method_kwargs={}),
	categorical_encoders={'__group_id__DE': NaN

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | MAE                             | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 7.3 K 
3  | prescalers                         | ModuleDict                      | 96    
4  | static_variable_selection          | VariableSelectionNetwork        | 1.8 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 3.1 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 2.4 K 
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K 
9  | static_context_initial_cell_l

Epoch 0: 100%|██████████| 50/50 [00:32<00:00,  1.53it/s, v_num=38, train_loss_step=0.482]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 50/50 [00:32<00:00,  1.55it/s, v_num=38, train_loss_step=0.348, val_loss=0.939, train_loss_epoch=0.619]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 50/50 [00:32<00:00,  1.53it/s, v_num=38, train_loss_step=0.315, val_loss=0.999, train_loss_epoch=0.414]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 50/50 [00:32<00:00,  1.53it/s, v_num=38, train_loss_step=0.271, val_loss=1.010, train_loss_epoch=0.347]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 50/50 [00:32<00:00,  1.54it/s, v_num=38, train_loss_step=0.232, val_loss=1.020, train_loss_epoch=0.282]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 50/50 [00:32<00:00,  1.54it/s, v_num=38, train_loss_step=0.220, val_loss=0.997, train_loss_epoch=0.244]
Validation: |          | 0/? [

## 5. Calculate the Results

### Prepare test dataset

In [501]:
# First before comparing the predictions to the actuals we need to feed a modified version of our test dataset to the modelto get predictions
# In the following part some parameters should be manually adjusted as needed depending on the target
test_dataset = test_data_all_columns
columns_to_keep = ['DE_load_actual_entsoe_transparency'] # adjust as needed with target column
test_dataset = test_dataset.loc[:, columns_to_keep]

In [502]:
# Adding the same time features so we can use them as future known categoricals
test_dataset = add_time_features(test_dataset)

In [503]:
# Needed because TFT needs a group column
test_dataset['DE'] = 'DE'

In [504]:
# This is pytorch-forecasting syntax specific, the time_idx index should be continous so we set it continuing the time_idx index from train+val
test_dataset['time_idx'] = range(30536, 30536 + len(test_dataset)) 

In [505]:
# Drop the data index and delete this column
test_dataset=test_dataset.reset_index()
test_dataset=test_dataset.drop(columns='date')  

### Using the test dataset to feed information to the encoder and decoder in a rolling way to get predictions

In [ ]:
# Load the best model
best_tft = TemporalFusionTransformer.load_from_checkpoint('lightning_logs/lightning_logs/version_30/checkpoints/epoch=12-step=650.ckpt')# adjust as needed
# Define an empty dataframe to append all predictions to
all_df = pd.DataFrame()

max_encoder_length = 96  
max_prediction_length = 192 # adjust as needed
target_name = 'DE_load_actual_entsoe_transparency'# adjust as needed

# We need to stop our predictions max_encoder_length + max_prediction_length before the end of the test dataset otherwise we get errors
addition_var = max_encoder_length + max_prediction_length

for i in range(0, len(test_dataset)-addition_var, max_prediction_length):
    
    new_pred_collection = []
    # Define the data for the encoder and decoder
    # The encoder contains data from i until the max_encoder_length
    test_dataset_short = test_dataset.iloc[i : i + max_encoder_length]
    encoder_data = test_dataset_short[lambda x: x.time_idx > x.time_idx.max() - max_encoder_length]
    
    # The decoder contains information (here: known future time features) from the last point of the encoder until the prediction length
    test_dataset_decoder = test_dataset.iloc[i+ max_encoder_length : i + max_encoder_length + max_prediction_length]
    # Set the target to zero so the model is not spoiled
    test_dataset_decoder[target_name]=0.0
    decoder_data = test_dataset_decoder
    
    # Concatenate encoder and decoder information
    new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
    
    # Make predictions
    new_raw_predictions = best_tft.predict(new_prediction_data, mode="raw", return_x=True)

    # Reformatting and writing the current predictions in the overall all_df
    for i in range (test_dataset.shape[1]-8): # Number of columns - 8 because our data has 8 time features
        new_pred = (new_raw_predictions[0][0][i].cpu().detach().numpy())
        new_pred = new_pred.squeeze()
        new_pred_collection.append(new_pred)
        
    local_df = pd.DataFrame(new_pred_collection).transpose()
    all_df = pd.concat([all_df, local_df])

### Calculate MAE and MSE

In [508]:
# The original test_dataset and the all_df with all our predictions do not have the same size yet
# This is for two main reasons 1) The predictions only start after one full length of encoder input of size (96), therefore we cut off the first 96 rows
# Additionally since we stop max_encoder_length + max_prediction_length steps before the end of test dataset due to pytorch-forecasting TFT implementation,
# we need to stop early and cut-off in the end as well 

def calc_mse_mae(test_dataset, all_df,target_name,max_prediction_length):
    # Cut off at the start
    test_dataset_2=test_dataset[96:]

    # Cut off at the end
    length_all_df = len(all_df)
    length_test_dataset_2 = len(test_dataset_2)
    difference_to_substract = length_test_dataset_2-length_all_df
    test_dataset_2 = test_dataset_2.iloc[:-difference_to_substract] 
    test_dataset_2= test_dataset_2.iloc[:, 0]
    all_df_tensor = torch.tensor(all_df.values)
    
    torch_test_dataset = torch.tensor(test_dataset_2.values)
    torch_test_dataset=torch_test_dataset.reshape(-1, 1) # reshape to match size

    print(torch_test_dataset.size())
    print(all_df_tensor.size())
    
    # Save the tensors for the seasonality plots
    torch.save(all_df_tensor, f'TFT_Results/saved_predictions_univariate/predictions {target_name}{max_prediction_length}.pt')
    torch.save(torch_test_dataset, f'TFT_Results/saved_predictions_univariate/true {target_name}{max_prediction_length}.pt')
    
    mae = F.l1_loss(all_df_tensor, torch_test_dataset)
    # Print the MAE
    print("MAE:", mae)

    # Calculate the MSE
    mse = F.mse_loss(all_df_tensor, torch_test_dataset)
    # Print the MSE
    print("MSE:", mse)

In [509]:
calc_mse_mae(test_dataset, all_df,target_name, max_prediction_length)

torch.Size([8640, 1])
torch.Size([8640, 1])
MAE: tensor(0.2150, dtype=torch.float64)
MSE: tensor(0.0954, dtype=torch.float64)
